# Исследование надежности заемщиков.

Кредитный отдел банка хочет построить модель кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит. Для этого нужно разобраться, какие факторы влияют на факт погашения кредита в срок, на основе данных со статистикой банка о платёжеспособности клиентов. 

**Цель исследования** - дать ответы на следующие вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Данные о платёжеспособности клиентов получим из файла `data.csv`. О качестве данных ничего не известно. Поэтому сначала понадобится обзор данных. 

Проверим данные на ошибки и оценим их влияние на исследование. Затем проведем предобработку данных и исправим критичные ошибки. После этого можно будет перейти к исследованию взаимосвязей.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Получение ответов на вопросы.


## Обзор данных
Составим первое представление о данных банка.

Импортируем библиотеку `pandas`.

In [1]:
import pandas as pd # импорт библиотеки pandas

Прочитаем файл со статистикой о платёжеспособности клиентов `data.csv` из папки `/datasets` и сохраним его в переменной `data`:

In [2]:
data = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в data

Выведем на экран первые десять строк таблицы:

In [3]:
data.head(10)  # получение первых 10 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице:

In [4]:
data.info() # получение общей информации о данных в таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице с данными 12 столбцов. 

Поскольку количество значений в столбцах различается, можно сделать вывод, что в данных есть пропущенные значения.

Среди типов данных встречаются как строки `object`, так и целые `int64` и дробные `float64` числа.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.


Посмотрим, какие значения встречаются в каждом из столбцов.

In [5]:
# перечень названий столбцов таблицы data
columns = data.columns 
# вывод количества уникальных значений каждого столбца
for column in columns:
    print(f'столбец {column}')
    print(data[column].value_counts().sort_index())
    print()

столбец children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

столбец days_employed
-18388.949901     1
-17615.563266     1
-16593.472817     1
-16264.699501     1
-16119.687737     1
                 ..
 401663.850046    1
 401674.466633    1
 401675.093434    1
 401715.811749    1
 401755.400475    1
Name: days_employed, Length: 19351, dtype: int64

столбец dob_years
0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58

Видно, что попадаются странные значения в столбцах `children`, `dob_years`, `gender`. В колонке `days_employed` данные варьируются от отрицательных до слишком больших. В информации об образовании присутствуют дубликаты. В столбце `purpose` много одинаковых по смыслу значений, написанных по-разному. Также есть столбцы, смысл которых дублирует друг друга (`education` и `education_id`, `family_status` и `family_status_id`). Всё это исправим на следующем этапе. 

**Выводы**

В каждой строке таблицы — данные о клиенте банка: его пол, возраст, семейное положение, количество детей, образование, тип занятости, трудовой стаж, ежемесячный доход, цель получения кредита, а также была ли у клиента задолженность по кредиту.

Предварительно можно утверждать, что данных достаточно для проведения анализа. Но необходима их предобработка, так как есть пропуски значений, дубликаты (например, в столбце `education`), аномальные значения (например, отрицательный трудовой стаж). При дальнейшем исследовании могут быть обнаружены и другие проблемы.

Следующим шагом станет предобработка данных.

## Предобработка данных

### Заполнение пропусков
Сначала посчитаем, сколько в таблице пропущенных значений:

In [6]:
data.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Обнаружены пропуски в данных о трудовом стаже и ежемесячном доходе. Их количество совпадает. Возможно, данные пропущены в одних и тех же строках.

Найдём все строки с пропусками в столбце `days_employed` и просмотрим первые десять.

In [7]:
data[data['days_employed'].isna()].head(10)  # просмотр строк с пропущенными значениями

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Похоже, пропуски действительно встречаются в одних и тех же строках. Это могло произойти, например, если клиент не указал эти данные, работает неофициально или является безработным.

Проверим, какую долю от общего числа наблюдений составляют пропущенные значения.

In [8]:
# расчет доли пропущенных значений
empty_values = data['days_employed'].isna().sum() / data.shape[0]
print(f'Доля пропущенных значений: {empty_values:.1%}')

Доля пропущенных значений: 10.1%


10% пропусков - это довольно много. Вместо того чтобы удалять строки, заполним пропуски характерными значениями. Лучше всего воспользоваться медианой, поскольку она менее чувствительна к аномалиям.

На предыдущем шаге мы выяснили, что в столбце `days_employed` есть отрицательные и аномально большие значения. Возможно, стоит сначала разобраться с ними, а потом заполнять пропуски, чтобы правильно рассчитать медиану.

Заполним пока пропуски в столбце `total_income`. Логично предположить, что ежемесячный доход клиента зависит от его типа занятости. Проверим, различаются ли медианы для клиентов из разных групп с помощью сводной таблицы.

In [9]:
# построение сводной таблицы
data_pivot = data.pivot_table(index='income_type', values='total_income', aggfunc='median') 
display(data_pivot)

,total_income
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


Действительно, серединные значения ежемесячного дохода у клиентов с разным типом занятости различаются. Заменим пропуски в `total_income` на соответствующую медиану.

In [10]:
# создание списка с уникальными значениями столбца income_type
income_types = data['income_type'].unique() 

# для каждого уникального значения рассчитываем медиану дохода и заменяем ею пропуски
for type in income_types: 
    median_income_type = data[data['income_type'] == type]['total_income'].median() 
    data.loc[(data['income_type'] == type) & (data['total_income'].isna()),'total_income'] = median_income_type

Убедимся, что в `total_income` не осталось пропусков.

In [11]:
data['total_income'].isna().sum() # подсчёт пропусков

0

### Проверка данных на аномалии и исправления.

Обработаем аномальные значения в столбцах, выявленные в ходе обзора данных.
Среди данных о количестве детей встречаются странные значения `20` и `-1`. Вполне возможно, что `-1` - это ошибка занесения, связанная с постановкой тире. Заменим эти значения на `1`. Остается загадкой число `20`, это может быть как `0`, так и `2`. Поскольку доля этого значения невелика (76 из 21525), а некорректное заполнение может сильно повлиять на результаты исследования, стоит удалить эти строки.

In [12]:
data['children'] = data['children'].replace(-1,1)  # замена значений -1 на 1
data = data[data['children'] != 20].reset_index(drop=True)  # удаление строк со значением 20
data['children'].value_counts()  # проверка 

0    14149
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

В столбце `dob_years` обнаружено 100 строк с нулевым возрастом. Скорее всего, это неявные пропуски. Можно заменить их средними или медианными значениями. При этом стоит учесть, к какой категории относится клиент. Ведь средний возраст студента будет сильно отличаться от среднего возраста пенсионера. 

In [13]:
# построение сводной таблицы
data_pivot = data.pivot_table(index='income_type',values='dob_years',aggfunc='median') 
display(data_pivot)

,dob_years
income_type,
безработный,38.0
в декрете,39.0
госслужащий,40.0
компаньон,39.0
пенсионер,60.0
предприниматель,42.5
сотрудник,39.0
студент,22.0


Резко выбиваются только медианы у категорий `студент` и `пенсионер`. Все остальные находятся примерно в одном возрасте. Для верности заменим нулевой возраст клиента на соответствующую его типу занятости медиану.

In [14]:
# создание списка с уникальными значениями столбца income_type
income_types = data['income_type'].unique() 

# для каждого уникального значения рассчитываем медиану дохода и заменяем ею нулевой возраст
for type in income_types:
    median_years_type = data[data['income_type'] == type]['dob_years'].median()
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == type),'dob_years'] = median_years_type

data['dob_years'].value_counts().sort_index() 

19.0     14
20.0     51
21.0    110
22.0    183
23.0    253
24.0    263
25.0    356
26.0    407
27.0    491
28.0    503
29.0    543
30.0    537
31.0    558
32.0    508
33.0    579
34.0    600
35.0    615
36.0    553
37.0    533
38.0    597
39.0    646
40.0    611
41.0    605
42.0    594
43.0    511
44.0    545
45.0    494
46.0    472
47.0    480
48.0    537
49.0    505
50.0    511
51.0    447
52.0    483
53.0    458
54.0    478
55.0    442
56.0    482
57.0    459
58.0    461
59.0    442
60.0    396
61.0    354
62.0    351
63.0    269
64.0    264
65.0    194
66.0    183
67.0    167
68.0     99
69.0     84
70.0     65
71.0     58
72.0     33
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

В столбце `gender` было обнаружено интересное значение: помимо женского и мужского пола в данных присутствует `XNA`. Такая строчка всего одна, удалим её.

In [15]:
data = data[data['gender'] != 'XNA'].reset_index(drop=True) # удаление строки с аномальным значением
data['gender'].value_counts() # проверка 

F    14189
M     7259
Name: gender, dtype: int64

Теперь обратимся к столбцу `days_employed`. 

In [16]:
data['days_employed'].describe() # вывод описательной статистики

count     19283.000000
mean      63144.630293
std      140913.145995
min      -18388.949901
25%       -2747.910655
50%       -1204.038231
75%        -290.174689
max      401755.400475
Name: days_employed, dtype: float64

Более 75% значений в нём отрицательные. Это может быть как ошибкой занесения (тире воспринято как минус), так и какой-то ошибкой в формуле расчёта. Такие значения мы можем привести к положительным. 

Также странным выглядит максимальное значение в ряду. 401 755 дней - это примерно 1100 лет. Совершенно непонятно, откуда могла взяться такая цифра. Сгруппируем данные по типу занятости и выведем описательную статистику.

In [17]:
# вывод описательной статистики стажа по типу занятости
data.groupby('income_type').agg({'days_employed':['min', 'max', 'mean']}) 

days_employed                              
                           min            max           mean
income_type                                                 
безработный      337524.466835  395302.838654  366413.652744
в декрете         -3296.759962   -3296.759962   -3296.759962
госслужащий      -15193.032201     -39.954170   -3398.915807
компаньон        -17615.563266     -30.195337   -2116.580391
пенсионер        328728.720605  401755.400475  365011.937155
предприниматель    -520.848083    -520.848083    -520.848083
сотрудник        -18388.949901     -24.141633   -2325.075758
студент            -578.751554    -578.751554    -578.751554

Интересное наблюдение: аномально высокие показатели трудового стажа замечены у пенсионеров и безработных, т.е. у тех людей, у которых нет текущего места работы. Возможно, оно учитывается в формуле расчета трудового стажа, и из-за его отсутствия появляются аномалии. Подробнее нам могли бы объяснить сотрудники банка, предоставившие исходные данные, но в рамках проекта мы не можем к ним обратиться.

Данные трудового стажа не нужны нам в дальнейшем исследовании, поэтому наилучшим выходом видится удаление этого столбца.

In [18]:
data = data.dropna(axis=1) # удаление столбца

###  Изменение типов данных.

Заменим вещественный тип данных в столбце `total_income` на целочисленный для ускорения расчетов. К тому же не имеет смысла проводить денежные расчеты с точностью до нескольких знаков после запятой.

In [19]:
# изменение типа данных 
data['total_income'] = data['total_income'].astype('int') 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21448 entries, 0 to 21447
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21448 non-null  int64  
 1   dob_years         21448 non-null  float64
 2   education         21448 non-null  object 
 3   education_id      21448 non-null  int64  
 4   family_status     21448 non-null  object 
 5   family_status_id  21448 non-null  int64  
 6   gender            21448 non-null  object 
 7   income_type       21448 non-null  object 
 8   debt              21448 non-null  int64  
 9   total_income      21448 non-null  int32  
 10  purpose           21448 non-null  object 
dtypes: float64(1), int32(1), int64(4), object(5)
memory usage: 1.7+ MB


### Удаление дубликатов.

Ранее мы обнаружили, что в столбце `education` присутствуют неявные дубликаты. Часть значений написана заглавными буквами, часть строчными, часть начинается с заглавной буквы. Это могло произойти из-за того, что информация заносилась разными людьми, или в банке нет стандартов по оформлению данных. 

Приведем все значения в столбце к нижнему регистру.

In [20]:
data['education'] = data['education'].str.lower() # приведение данных к одному регистру
data['education'].value_counts() # проверка

среднее                15173
высшее                  5246
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь с образованием всё в порядке. 

Проверим, есть ли в таблице строки-дубликаты.

In [21]:
data.duplicated().sum() # подсчёт явных дубликатов

71

Найден 71 дубликат. Скорее всего, какие-то клиенты попали в базу не один раз. Удалим дубликаты, переиндексировав строки.

In [22]:
data = data.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Сделаем исходную таблицу более компактной и удобной для анализа. Как можно было заметить при обзоре данных, есть столбцы, смысл которых дублирует друг друга, при этом один столбец представляет собой значение, а другой - его идентификатор. Из этих столбцов мы можем сделать отдельные "словари", а в исходной таблице оставить только идентификаторы.

In [23]:
# создание словаря для образования
education_dict = data[['education_id','education']].drop_duplicates().reset_index(drop = True) 
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [24]:
# создание словаря для семейного статуса
family_status_dict = data[['family_status_id','family_status']].drop_duplicates().reset_index(drop = True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [25]:
data = data.drop(columns = ['education', 'family_status']) # удаление колонок из исходного датафрейма

In [26]:
data.columns # вывод названий столбцов

Index(['children', 'dob_years', 'education_id', 'family_status_id', 'gender',
       'income_type', 'debt', 'total_income', 'purpose'],
      dtype='object')

### Категоризация дохода.

Информация о доходах клиентов представлена множеством значений. В таком виде сложно проводить анализ и выявлять какие-либо взаимосвязи. Поэтому проведем категоризацию данных.

Создадим столбец `total_income_category` с категориями:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

Для этого создадим функцию, которая будет проверять значения и определять их в нужную категорию, а затем применим её к столбцу `total_income`.

In [27]:
# Функция для категоризации доходов
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

# добавление столбца с категориями в data
data['total_income_category'] = data['total_income'].apply(total_income_category) 

# вывод первых 10 строк
data.head(10) 

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,27.0,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,43.0,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,50.0,1,0,M,сотрудник,0,135823,образование,C
8,2,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Категоризация целей кредита.

Осталось разобраться со столбцом `purpose`, в котором собраны цели, на которые выдается кредит. Снова выведем уникальные значения этого столбца.

In [28]:
(data['purpose'].value_counts().sort_index()) #вывод количества уникальных значений

автомобили                                477
автомобиль                                493
высшее образование                        447
дополнительное образование                457
жилье                                     642
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     470
на покупку подержанного автомобиля        472
на покупку своего автомобиля              505
на проведение свадьбы                     764
недвижимость                              632
образование                               445
операции с жильем                         648
операции с коммерческой недвижимостью     646
операции с недвижимостью                  674
операции со своей недвижимостью           626
покупка жилой недвижимости                604
покупка жилья                             643
покупка жилья для сдачи                   650
покупка жилья для семьи                   637
покупка коммерческой недвижимости 

Несмотря на кажущееся разнообразие значений, их можно разделить на 4 категории:

* операции с автомобилем,

* операции с недвижимостью,

* проведение свадьбы,

* получение образования.

Поступим так же, как на предыдущем шаге: создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`.

In [29]:
# Функция для категоризации целей кредита
def purpose_category(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образ' in purpose:
        return 'получение образования'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'жил' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    return 'категория не определена'

# добавление столбца с категориями в data
data['purpose_category'] = data['purpose'].apply(purpose_category) 

# вывод первых 10 строк
data.head(10) 

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,27.0,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,43.0,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,50.0,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


**Выводы**

При предобработке данных были устранены следующие проблемы: 
* Заполнены пропуски в данных о ежемесячных доходах;
* Заполнены неявные пропуски у возраста клиентов;
* Удалены или исправлены аномальные количества детей и несуществующий пол;
* Удален столбец с трудовым стажем как имеющий странные значения и не влияющий на проверку гипотез;
* Удалены явные и исправлены неявные дубликаты;
* Изменен тип данных на целочисленный в столбце с ежемесячными доходами, а также проведена его категоризация; 
* Разделены на категории цели получения кредита;
* Созданы дополнительные таблицы-"словари" для образования и семейного положения.

Благодаря проделанной работе исследование станет более точным.

Теперь можно проверить, какие факторы влияют на факт погашения кредита в срок.

## Ответы на вопросы.

### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для исследования зависимости между количеством детей и возвратом кредита срок сгруппируем исходную таблицу и посчитаем количество и долю просрочек в каждой группе.

In [30]:
# группировка и вывод количества и суммы значений в столбце debt
data_analysis_children = data.groupby(['children']).agg({'debt':['sum','count']}) 

# расчет доли просрочек
data_analysis_children[('debt','ratio')] = (
    data_analysis_children[('debt','sum')] / data_analysis_children[('debt','count')]
)

# вывод отсортированной таблицы
data_analysis_children.sort_values(by=('debt','ratio'), ascending=False) 

debt                 
           sum  count     ratio
children                       
4            4     41  0.097561
2          194   2052  0.094542
1          445   4855  0.091658
3           27    330  0.081818
0         1063  14090  0.075444
5            0      9  0.000000

**Вывод** 

Совсем не допускают просрочек клиенты с пятью детьми. Но их общее количество в выборке слишком мало, чтобы считать такой результат значимым. Наименьшая доля просрочек среди значимых результатов у бездетных (7,5%). Среди людей с детьми четкой зависимости между количеством детей и частотой невозвратов кредитов не прослеживается. Так, например, семьи с тремя детьми выплачивают кредиты чаще, чем с одним ребенком.

Таким образом, можно сказать, что имеет значение только есть ребенок или нет, а их количество на возврат долга не влияет.

### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Аналогичным способом исследуем зависимость между семейным положением и возвратом кредита в срок. При этом для наглядности нужно будет добавить в сгруппированную таблицу значения `family_status` из ранее созданного словаря.

In [31]:
# группировка и вывод количества и суммы значений в столбце debt
data_analysis_family = data.groupby(['family_status_id']).agg({'debt':['sum','count']}) 

# расчет доли просрочек
data_analysis_family[('debt','ratio')] = data_analysis_family[('debt','sum')] / data_analysis_family[('debt','count')] 

# объединение получившейся с таблицы со словарем
data_analysis_family = data_analysis_family.merge(right=family_status_dict, on='family_status_id', how='left') 

# вывод отсортированной таблицы
data_analysis_family.sort_values(by=('debt','ratio'), ascending=False) 

D:\Anaconda3\envs\praktikum\lib\site-packages\pandas\core\frame.py:9203: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  validate=validate,


,family_status_id,"(debt, sum)","(debt, count)","(debt, ratio)",family_status
4,4,273,2801,0.097465,Не женат / не замужем
1,1,385,4138,0.093040,гражданский брак
0,0,928,12290,0.075509,женат / замужем
3,3,84,1193,0.070411,в разводе
2,2,63,955,0.065969,вдовец / вдова


Самая большая доля просрочек у клиентов, не побывавших в браке (9,7%) и живущих в гражданском браке (9,3%). Это можно объяснить тем, что у одиноких людей в сложной жизненной ситуации нет поддержки от супруга. Кроме того, скорее всего это по большей части молодые люди, возможно, чуть менее ответственные или не имеющие стабильного источника доходов. 

Наиболее надежными заемщиками оказались вдовцы. Они не возвращают кредиты в срок лишь в 6,6% случаев. 

Можно утверждать, что семейное положение влияет на возврат кредита в срок.

### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Теперь проверим, как влияет уровень дохода на надежность заемщиков.

In [32]:
# группировка и вывод количества и суммы значений в столбце debt
data_analysis_income = data.groupby(['total_income_category']).agg({'debt':['sum','count']}) 

# расчет доли просрочек
data_analysis_income[('debt','ratio')] = data_analysis_income[('debt','sum')] / data_analysis_income[('debt','count')] 

# вывод отсортированной таблицы
data_analysis_income.sort_values(by=('debt','ratio'), ascending=False) 

debt                 
                        sum  count     ratio
total_income_category                       
E                         2     22  0.090909
C                      1354  15959  0.084842
A                         2     25  0.080000
B                       354   5021  0.070504
D                        21    350  0.060000

Самой рискованной группой клиентов для банка являются люди с доходом до 30 000 рублей в месяц, что логично, но они и оформляют кредит реже. На втором месте "средний класс", с доходами от 50 000 до 200 000 рублей в месяц. 

Наиболее надежные заемщики - люди с доходами от  30 000 до 50 000 рублей, они допускают просрочку в 6% случаев.

Таким образом, между уровнем дохода и возвратом кредита в срок есть зависимость, но нелинейная.

### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Осталось узнать, есть ли зависимость между целью выдачи кредита и его невозвратом в срок.

In [33]:
# группировка и вывод количества и суммы значений в столбце debt
data_analysis_purpose = data.groupby(['purpose_category']).agg({'debt':['sum','count']}) 

# расчет доли просрочек
data_analysis_purpose[('debt','ratio')] = data_analysis_purpose[('debt','sum')] / data_analysis_purpose[('debt','count')] 

# вывод отсортированной таблицы
data_analysis_purpose.sort_values(by=('debt','ratio'), ascending=False)

debt                 
                          sum  count     ratio
purpose_category                              
операции с автомобилем    401   4290  0.093473
получение образования     369   3998  0.092296
проведение свадьбы        183   2315  0.079050
операции с недвижимостью  780  10774  0.072397

Наименее надежными клиентами оказались те, кто берут кредит на операции с автомобилем или получение образования. Они не возвращают в срок кредит более, чем в 9% случаев.

Наиболее ответственно относятся к кредиту заемщики, берущие кредит на проведение операций с недвижимостью.

## Общий вывод:

В ходе исследования было установлено следующее.

Лучше возвращают кредиты клиенты без детей. Количество же детей на возврат долга не влияет.

Между семейным положением и платежеспособностью есть взаимосвязь: холостые возвразают кредиты хуже, чем побывавшие в браке.

Хуже всего выплачивают кредит клиенты с низкими доходами, но прямой зависимости между ростом доходов и улучшением платежоной дисциплины нет.

Среди целей выдачи кредита наиболее рискованными являются операции с автомобилем или получение образования, наименее - операции с недвижимостью.